In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, KFold

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import  roc_auc_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import category_encoders as ce
from sklearn.pipeline import Pipeline


In [2]:
enzy = pd.read_csv('Train.csv')
#train, test = train_test_split(enzy, test_size = 0.2, random_state = 101)
train = enzy.sample(400000)
trainX = train.drop('LABEL', axis=1)
trainy = train['LABEL']

# testX = test.drop('LABEL', axis=1)
# testy = test['LABEL']

In [3]:
enztest = pd.read_csv('test.csv')
enzytest = enztest.copy()

In [4]:
def Extract(f):    
    for p in ['A','R','N','D','B','C','Q','E','Z','G','H','I','L','K','M','F','P','S','T','W','Y','V']:
        def counter(sequence, let):
            return sequence.count(let)
        f[p]= f.apply(lambda x: counter(x.SEQUENCE, p), axis =1)
    f['Length'] = f.apply(lambda x: len(x.SEQUENCE), axis =1)
    f['other'] = f['Length'] - (f['A']+f['R']+f['N']+f['D']+f['B']+f['C']+f['Q']+f['E']+f['Z']+f['G']+f['H']+f['I']+f['L']+f['K']+f['M']+f['F']+f['P']+f['S']+f['T']+f['W']+f['Y']+f['V'])
    f.drop(['SEQUENCE','SEQUENCE_ID'], axis=1, inplace = True)

Extract(trainX)
#Extract(testX)
Extract(enzytest)

In [5]:
# mt = Pipeline([('Encode', ce.BinaryEncoder(cols= 'CREATURE')),('Sc',RobustScaler())])
# trainX= mt.fit_transform(trainX)
# #testX= mt.transform(testX)
# enzytest= mt.transform(enzytest)

In [6]:
final = Pipeline([
('Encode', ce.BinaryEncoder(cols= 'CREATURE')),
('Standization', RobustScaler()),
('Ensemble' , VotingClassifier([('rfc', RandomForestClassifier(max_depth=100, max_features='sqrt', min_samples_leaf=2, min_samples_split=10, n_estimators=500)), 
                                ('knc', KNeighborsClassifier(algorithm = 'ball_tree', leaf_size = 13, n_neighbors = 18, weights = 'distance'))])
)])

final.fit(trainX, trainy)

Pipeline(steps=[('Encode', BinaryEncoder(cols='CREATURE')),
                ('Standization', RobustScaler()),
                ('Ensemble',
                 VotingClassifier(estimators=[('rfc',
                                               RandomForestClassifier(max_depth=100,
                                                                      max_features='sqrt',
                                                                      min_samples_leaf=2,
                                                                      min_samples_split=10,
                                                                      n_estimators=500)),
                                              ('knc',
                                               KNeighborsClassifier(algorithm='ball_tree',
                                                                    leaf_size=13,
                                                                    n_neighbors=18,
                                                

In [7]:
pred = final.predict(enzytest)

In [8]:
one = pd.DataFrame(enztest['SEQUENCE_ID'])
two = pd.DataFrame(pred)
Done = pd.concat([one,two], axis=1)
Done.columns = ['SEQUENCE_ID','LABEL']
Done.to_csv('Enzyme-Solution.csv',index=False)

In [ ]:
from keras.layers import Dense, Activation
from keras.models import Sequential
#from keras.wrappers.scikit_learn import KerasClassifier

#def model():
md = Sequential()
md.add(Dense(32, activation= 'relu', input_dim = trainX.shape[1]))
md.add(Dense(units = 32, activation= 'relu'))
md.add(Dense(units = 32, activation= 'relu'))
md.add(Dense(units = 32, activation= 'softmax'))
md.add(Dense(units = 20,))

md.compile(optimizer='adam', loss= 'categorical_crossentropy', metrics=['accuracy'])
md.fit(trainX, trainy, batch_size=trainX.shape[1], epochs=50, verbose=0)
#return md
    
# estimator = KerasClassifier(build_fn=model, epochs=200, batch_size=5, verbose=0)
# kfold = KFold(n_splits=10, shuffle=True)
# results = cross_val_score(estimator, trainX, trainy, cv=kfold)
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

ypred = md.predict(testX)
print(roc_auc_score(testy, ypred))

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin
class DropFeatures(BaseEstimator, TransformerMixin):

    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # Drop irrelevant features
        X.drop(self.attribute_names, axis=1, inplace=True)
        return X

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin
class Extract(BaseEstimator, TransformerMixin):

    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        for p in ['A','R','N','D','B','C','Q','E','Z','G','H','I','L','K','M','F','P','S','T','W','Y','V']:
            def counter(sequence, let):
                return sequence.count(let)
            X[p]= X.apply(lambda self: counter(self.SEQUENCE, p), axis =1)
        X['Length'] = X.apply(lambda self: len(self.SEQUENCE), axis =1)
        X['other'] = X['Length'] - (X['A']+X['R']+X['N']+X['D']+X['B']+X['C']+X['Q']+X['E']+X['Z']+X['G']+X['H']+X['I']+X['L']+X['K']+X['M']+X['X']+X['P']+X['S']+X['T']+X['W']+X['Y']+X['V'])
        X.drop(self.attribute_names, axis=1, inplace=True)
        return X


In [11]:
enzy = pd.read_csv('Train.csv', chunksize =100000, iterator=True)

for enz in enzy:
    def Extract(f):    
        for p in ['A','R','N','D','B','C','Q','E','Z','G','H','I','L','K','M','F','P','S','T','W','Y','V']:
            def counter(sequence, let):
                return sequence.count(let)
            f[p]= f.apply(lambda x: counter(x.SEQUENCE, p), axis =1)
        f['Length'] = f.apply(lambda x: len(x.SEQUENCE), axis =1)
        f['other'] = f['Length'] - (f['A']+f['R']+f['N']+f['D']+f['B']+f['C']+f['Q']+f['E']+f['Z']+f['G']+f['H']+f['I']+f['L']+f['K']+f['M']+f['F']+f['P']+f['S']+f['T']+f['W']+f['Y']+f['V'])
        f.drop(['SEQUENCE','SEQUENCE_ID'], axis=1, inplace = True)

    Extract(enz)

    X = enz.drop('LABEL', axis=1)
    y = enz['LABEL']

    Encode = ce.BinaryEncoder(cols= 'CREATURE')
    X = Encode.fit_transform(X,)

    sc = StandardScaler()
    X = sc.fit_transform(X)

    # Finding the BaseLine perfomance of the various models
    # prepare models
    models = []

    # Adding algorthms
    #models.append(('sgd', SGDClassifier()))
    models.append(('knc', KNeighborsClassifier()))
    models.append(('rfc', RandomForestClassifier()))
    #models.append(('xg', XGBClassifier()))
    #models.append(('svc', SVC()))
    #models.append(('mlp', MLPClassifier()))

    # evaluate -cross validation- each model in turn
    results = []
    names = []
    #scoring = ['f1_weighted', 'roc_auc_ovo_weighted']
    scoring = 'f1_weighted'
    for name, model in models:
        kfold = KFold(n_splits=7, random_state=7)
        #cv_results = cross_validate(model, X, y, cv=kfold, scoring=scoring, n_jobs=-1, return_train_score=True)
        cv_results = cross_val_score(model, X, y, cv=kfold, scoring=scoring, n_jobs=-1)
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f" % (name, cv_results.mean())
        print(msg)
        #msg = "%s: %f (%f)" % (name, cv_results['train_f1_weighted'].mean(), cv_results['test_roc_auc_ovo_weighted'].mean())
results.append(results.mean())
names.append(name)
msg = "%s: %f" % (name, results.mean())
print(msg)

knc: 0.805325
rfc: 0.816907
knc: 0.802734
rfc: 0.816063
knc: 0.801359
rfc: 0.811419
knc: 0.804776
rfc: 0.814156
knc: 0.802506
rfc: 0.813193
knc: 0.805005
rfc: 0.815938
knc: 0.802699
rfc: 0.812626
knc: 0.801287
rfc: 0.812729
knc: 0.758673
rfc: 0.789016


AttributeError: 'list' object has no attribute 'mean'

In [12]:
    # importing libraries for bayesian optimization on Random Forest
    from skopt import BayesSearchCV
    from skopt.space import Real, Categorical, Integer

    search_space = {'max_depth': Integer(10, 100), 'n_estimators':Integer(200,500), "max_features": Categorical(['auto', 'sqrt','log2']), "min_samples_leaf": Integer(2, 10), "min_samples_split": Integer(2, 10)}

    rfr_bayes = BayesSearchCV(RandomForestClassifier(), search_space, n_iter=32, scoring='f1_weighted', n_jobs=-1, cv =5)
    rfr_bayes.fit(X, y)


BayesSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=32, n_jobs=-1,
              scoring='f1_weighted',
              search_spaces={'max_depth': Integer(low=10, high=100, prior='uniform', transform='identity'),
                             'max_features': Categorical(categories=('auto', 'sqrt', 'log2'), prior=None),
                             'min_samples_leaf': Integer(low=2, high=10, prior='uniform', transform='identity'),
                             'min_samples_split': Integer(low=2, high=10, prior='uniform', transform='identity'),
                             'n_estimators': Integer(low=200, high=500, prior='uniform', transform='identity')})

In [13]:
# importing libraries for bayesian optimization on KNN
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

search_space = {'n_neighbors': Integer(1,200), 'weights': Categorical(['uniform', 'distance']), "algorithm": Categorical(['auto', 'ball_tree', 'kd_tree', 'brute']), "leaf_size": Integer(2, 200)}
 
kn_bayes = BayesSearchCV(KNeighborsClassifier(), search_space, n_iter=32, scoring='f1_weighted', n_jobs=-1, cv =5)
kn_bayes.fit(X, y)


MemoryError: Unable to allocate 1.00 GiB for an array with shape (2854, 47022) and data type float64

In [ ]:
vt = VotingClassifier([('rfc',rfr_bayes.best_estimator_), ('knc', kn_bayes.best_estimator_)])
kfold = KFold(n_splits=10, random_state=7)
#cv_results = cross_validate(model, X, y, cv=kfold, scoring=scoring, n_jobs=-1, return_train_score=True)
cv_results = cross_val_score(vt, X, y, cv=5, scoring='f1_weighted', n_jobs=-1)
#msg = "%s: %f (%f)" % (name, cv_results['train_f1_weighted'].mean(), cv_results['test_roc_auc_ovo_weighted'].mean())
msg = "%s: %f" % ('vt', cv_results.mean())
print(msg)